In [1]:
!pip install pymatgen
!pip install mp_api
from pymatgen.core.composition import Composition
from pymatgen.core.periodic_table import Element
from pymatgen.core.structure import Structure
import pandas as pd
import numpy as np
import plotly as pt
import seaborn as sns
import requests
import json
import os
import plotly.express as px

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.6/55.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.9/51.9 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.3/332.3 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.9/119.9 kB 8.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 809.1/809.1 kB 38.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.1/60.1 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 753.1/753.1 kB 44.9 MB/s eta 0:00:00
  Created wheel for bibtexparser: filename=bibtexparser-1.4.3-py3-none-any.whl size=43549 sha256=898b3f2043032de377a6f0d35dea2be5286284c5e26b003814fcec8f02282403
  Stored in directory: /root/.cache/pip/wheels/1f/7d/e9/1ff2509f13767a55df1279744adfb757f4ab94b2cbe761f56a
Successfully built bibtexparser
   ━━━━━━━━━━━━━━━━━━━

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
df = pd.read_excel("/content/drive/MyDrive/University/Artificial intelligence in chemistry/Perovskite project/Perovskite-liked-oxides-bandgap-prediction/Data/Perovskite dataset.xlsx",sheet_name='Export')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 523 entries, 0 to 522
Data columns (total 62 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            523 non-null    object 
 1   Hill formula                          523 non-null    object 
 2   Interlayer space composition          3 non-null      object 
 3   Dopant                                53 non-null     object 
 4   Dopant, wt%                           42 non-null     float64
 5   Bandgap, eV                           523 non-null    float64
 6   DOI                                   518 non-null    object 
 7   Materials Project ID                  518 non-null    object 
 8   COD_ID                                518 non-null    object 
 9   Springer_ID                           57 non-null     object 
 10  Z                                     257 non-null    float64
 11  Z_MP               

In [7]:
df_dataset = df[['Perovskite','Bandgap, eV','avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       'frac s valence electrons', 'frac p valence electrons',
       'frac d valence electrons', 'frac f valence electrons',
       'MagpieData minimum Electronegativity',
       'MagpieData maximum Electronegativity',
       'MagpieData range Electronegativity',
       'MagpieData mean Electronegativity',
       'Average Pauling electronegativity',
       'Average Mulliken electronegativity',
       'Valence Electrons Density avg', 'Oxygen_concentration avg',
       'Packing fraction avg']]

In [8]:
df.columns


Index(['Perovskite', 'Hill formula', 'Interlayer space composition', 'Dopant',
       'Dopant, wt%', 'Bandgap, eV', 'DOI', 'Materials Project ID', 'COD_ID',
       'Springer_ID', 'Z', 'Z_MP', 'Z_COD', 'Z_Springer', 'a, A', 'b, A',
       'c, A', 'Symmetry group', 'd,A', 'a_MP', 'b_MP', 'c_MP', 'a_COD',
       'b_COD', 'c_COD', 'a_Springer', 'b_Springer', 'c_Springer',
       'Number of octahedrons on a layer', 'Valence electrons', 'Volume',
       'Volume_MP', 'Volume_COD', 'Volume_Springer',
       'Valence Electrons Density', 'Valence Electrons Density_MP',
       'Valence Electrons Density_COD', 'Springer_Valence Electrons Density',
       'avg s valence electrons', 'avg p valence electrons',
       'avg d valence electrons', 'avg f valence electrons',
       'frac s valence electrons', 'frac p valence electrons',
       'frac d valence electrons', 'frac f valence electrons',
       'MagpieData minimum Electronegativity',
       'MagpieData maximum Electronegativity',
       'Magpie

#Missing data handling

In [9]:
df_dataset.isna().sum()

,0
Perovskite,0
"Bandgap, eV",0
avg s valence electrons,5
avg p valence electrons,5
avg d valence electrons,5
avg f valence electrons,5
frac s valence electrons,5
frac p valence electrons,5
frac d valence electrons,5
frac f valence electrons,5


In [10]:
df_dataset = df_dataset.dropna()
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 518 entries, 0 to 517
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            518 non-null    object 
 1   Bandgap, eV                           518 non-null    float64
 2   avg s valence electrons               518 non-null    float64
 3   avg p valence electrons               518 non-null    float64
 4   avg d valence electrons               518 non-null    float64
 5   avg f valence electrons               518 non-null    float64
 6   frac s valence electrons              518 non-null    float64
 7   frac p valence electrons              518 non-null    float64
 8   frac d valence electrons              518 non-null    float64
 9   frac f valence electrons              518 non-null    float64
 10  MagpieData minimum Electronegativity  518 non-null    float64
 11  MagpieData maximum Elect

In [11]:
import matplotlib.pyplot as plt

In [12]:
for col in df_dataset.columns:
    fig = px.histogram(df_dataset, x=col, nbins=20, title=f'Histogram of {col}',width=800, height=500)
    fig.show()


In [13]:
#for col in df_dataset.select_dtypes(include='number').columns:
#    Q1 = df_dataset[col].quantile(0.25)
#    Q3 = df_dataset[col].quantile(0.75)
#    IQR = Q3 - Q1
#    outliers = df_dataset[(df_dataset[col] < Q1 - 1.5 * IQR) | (df_dataset[col] > Q3 + 1.5 * IQR)]
#    print(f"{col}: {len(outliers)} outliers")

In [14]:
min_boundary = {
    "Average Pauling electronegativity": 2.3,
    "Average Mulliken electronegativity": 5.3,
    "Valence Electrons Density avg": 0.025,
    "Oxygen_concentration avg": 0.01,
}
max_boundary = {
    "Average Pauling electronegativity": 3.5,
    "Average Mulliken electronegativity": 7,
    "Valence Electrons Density avg": 0.13,
    "Oxygen_concentration avg": 0.07,
}

In [15]:
for col in min_boundary:
  min = min_boundary[col]
  max = max_boundary[col]
  df_dataset = df_dataset[(df_dataset[col] >= min) & (df_dataset[col] <= max)]
  print( df_dataset.shape[0])

510
510
481
467


#Duplicates handling



In [16]:
duplicates = df_dataset[ df_dataset.duplicated(subset=['Perovskite'])]
print(duplicates.shape[0])

167


In [17]:
duplicates.to_excel('duplicates.xlsx')

In [18]:
df_dataset = df_dataset.drop_duplicates(subset=['Perovskite'], keep='first')

In [19]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 516
Data columns (total 19 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   Perovskite                            300 non-null    object 
 1   Bandgap, eV                           300 non-null    float64
 2   avg s valence electrons               300 non-null    float64
 3   avg p valence electrons               300 non-null    float64
 4   avg d valence electrons               300 non-null    float64
 5   avg f valence electrons               300 non-null    float64
 6   frac s valence electrons              300 non-null    float64
 7   frac p valence electrons              300 non-null    float64
 8   frac d valence electrons              300 non-null    float64
 9   frac f valence electrons              300 non-null    float64
 10  MagpieData minimum Electronegativity  300 non-null    float64
 11  MagpieData maximum Elect

#Data normalizatoin


In [20]:
from sklearn.preprocessing import MinMaxScaler

In [21]:
df_dataset.columns

Index(['Perovskite', 'Bandgap, eV', 'avg s valence electrons',
       'avg p valence electrons', 'avg d valence electrons',
       'avg f valence electrons', 'frac s valence electrons',
       'frac p valence electrons', 'frac d valence electrons',
       'frac f valence electrons', 'MagpieData minimum Electronegativity',
       'MagpieData maximum Electronegativity',
       'MagpieData range Electronegativity',
       'MagpieData mean Electronegativity',
       'Average Pauling electronegativity',
       'Average Mulliken electronegativity', 'Valence Electrons Density avg',
       'Oxygen_concentration avg', 'Packing fraction avg'],
      dtype='object')

In [22]:
#ML_columns = ['Average Pauling electronegativity',
#       'Average Mulliken electronegativity', 'Valence Electrons Density avg']
#ML_columns = ['Average Pauling electronegativity',
#       'Average Mulliken electronegativity', 'Valence Electrons Density avg','frac d valence electrons', 'MagpieData range Electronegativity']
ML_columns = ['avg s valence electrons',
       'avg p valence electrons', 'avg d valence electrons',
       'avg f valence electrons', 'frac s valence electrons',
       'frac p valence electrons', 'frac d valence electrons',
       'frac f valence electrons', 'MagpieData minimum Electronegativity',
       'MagpieData maximum Electronegativity',
       'MagpieData range Electronegativity',
       'MagpieData mean Electronegativity',
       'Average Pauling electronegativity',
       'Average Mulliken electronegativity', 'Valence Electrons Density avg',
       'Oxygen_concentration avg', 'Packing fraction avg']
scaler = MinMaxScaler()  # default range 0-1

df_normalized = df_dataset.copy()
df_normalized[ML_columns] = scaler.fit_transform(df_normalized[ML_columns])
df_normalized.head()

,Perovskite,"Bandgap, eV",avg s valence electrons,avg p valence electrons,avg d valence electrons,avg f valence electrons,frac s valence electrons,frac p valence electrons,frac d valence electrons,frac f valence electrons,MagpieData minimum Electronegativity,MagpieData maximum Electronegativity,MagpieData range Electronegativity,MagpieData mean Electronegativity,Average Pauling electronegativity,Average Mulliken electronegativity,Valence Electrons Density avg,Oxygen_concentration avg,Packing fraction avg
1,KLaNb2O7,3.20,0.522727,0.515152,0.153967,0.0,0.465691,0.875796,0.260011,0.0,0.02439,0.0,0.97561,0.392627,0.392627,0.331697,0.290815,0.290815,0.736527
2,RbLaNb2O7,3.35,0.522727,0.515152,0.153967,0.0,0.465691,0.875796,0.260011,0.0,0.02439,0.0,0.97561,0.392627,0.392627,0.325099,0.483768,0.483768,0.771248
3,CsLaNb2O7,3.30,0.522727,0.515152,0.153967,0.0,0.465691,0.875796,0.260011,0.0,0.00000,0.0,1.00000,0.388495,0.388495,0.314137,0.481698,0.481698,0.818672
4,KCa2Nb3O10,3.35,0.562500,0.500000,0.140173,0.0,0.490702,0.875796,0.239931,0.0,0.02439,0.0,0.97561,0.349753,0.349753,0.309096,0.570249,0.570249,0.384262
5,RbCa2Nb3O10,3.50,0.562500,0.500000,0.140173,0.0,0.490702,0.875796,0.239931,0.0,0.02439,0.0,0.97561,0.349753,0.349753,0.304560,0.550449,0.550449,0.798980


In [23]:
ML_columns.append('Bandgap, eV')
#df_normalized = df_normalized[Y_X_cols]
#df_normalized.info()
print(ML_columns)

['avg s valence electrons', 'avg p valence electrons', 'avg d valence electrons', 'avg f valence electrons', 'frac s valence electrons', 'frac p valence electrons', 'frac d valence electrons', 'frac f valence electrons', 'MagpieData minimum Electronegativity', 'MagpieData maximum Electronegativity', 'MagpieData range Electronegativity', 'MagpieData mean Electronegativity', 'Average Pauling electronegativity', 'Average Mulliken electronegativity', 'Valence Electrons Density avg', 'Oxygen_concentration avg', 'Packing fraction avg', 'Bandgap, eV']


In [24]:
df_normalized = df_normalized[ML_columns]
df_normalized.info()

<class 'pandas.core.frame.DataFrame'>
Index: 300 entries, 1 to 516
Data columns (total 18 columns):
 #   Column                                Non-Null Count  Dtype  
---  ------                                --------------  -----  
 0   avg s valence electrons               300 non-null    float64
 1   avg p valence electrons               300 non-null    float64
 2   avg d valence electrons               300 non-null    float64
 3   avg f valence electrons               300 non-null    float64
 4   frac s valence electrons              300 non-null    float64
 5   frac p valence electrons              300 non-null    float64
 6   frac d valence electrons              300 non-null    float64
 7   frac f valence electrons              300 non-null    float64
 8   MagpieData minimum Electronegativity  300 non-null    float64
 9   MagpieData maximum Electronegativity  300 non-null    float64
 10  MagpieData range Electronegativity    300 non-null    float64
 11  MagpieData mean Electron

#Final overview


In [25]:
for col in df_normalized.columns:
    fig = px.histogram(df_normalized, x=col, nbins=20, title=f'Histogram of {col}',width=800, height=500)
    fig.show()


In [26]:
sns.pairplot(df_normalized)
plt.show()

Output hidden; open in https://colab.research.google.com to view.

In [27]:
corr = df_normalized.corr()
fig = px.imshow(
    corr,
    text_auto=True,       # show correlation values on cells
    color_continuous_scale='RdBu_r',  # blue-red diverging colors
    zmin=-1, zmax=1       # set range for correlation
)
fig.update_layout(title="Correlation Heatmap", width=600, height=500)
fig.show()

In [28]:
df_normalized = df_normalized.reset_index()
df_normalized.to_excel('dataset_ready_to_ML.xlsx')